# Parameter tuning for regression models

## GridSearchCV

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV


### Setting up


In [ ]:
# Load data
df = pd.read_excel("./housing_data.xlsx")

# Extract data 
X = df[["LSTAT"]].values
y = df["MEDV"].values


# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# Standardize data
sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

# Classifier
svr = SVR()

In [ ]:
df

In [ ]:
pd.DataFrame(data=svr.get_params(), index=["param"]).T

### Define parameter sets

In [ ]:
param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]

set1 = {'C': param_range,
        'kernel': ['linear']}

set2 = {'C': param_range,
        'gamma': param_range,
        'kernel': ['rbf']}

param_grid = [set1, set2]

### GridSearchCV classifier

- Don't forget to change the scoring function.

In [ ]:
gs = GridSearchCV(estimator=svr, 
                  param_grid=param_grid, 
                  scoring='neg_mean_squared_error', 
                  cv=5,
                  n_jobs=-1)

In [ ]:
# Get parameter names
for k, v in gs.get_params().items():
    print(f"{k:35.35s}: {str(v)}")

### Training

In [ ]:
gs.fit(X_train_std,y_train)

In [ ]:
df = pd.DataFrame(gs.cv_results_)
print(df.shape)
display(df.head())

In [ ]:
df = df.sort_values(by=['rank_test_score'])
display(df.head())

### Refitting
- Note that grid search already refit the entire training data with the best parameters. You can check this from this setting.

In [ ]:
gs.refit

In [ ]:
print(gs.best_score_)
print(gs.best_params_)

In [ ]:
y_pred = gs.predict(X_test)
testing_accuracy = gs.score(X_test_std,y_test)
print(f"Testing accuracy: {testing_accuracy:6.3f}")